<a href="https://colab.research.google.com/github/polina-minaeva/collaborative-filtering/blob/main/3_%D0%9A%D0%BE%D0%BB%D0%BB%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%B0%D1%8F_%D1%84%D0%B8%D0%BB%D1%8C%D1%82%D1%80%D0%B0%D1%86%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Сделаем рекомендательную систему на основе коллаборативной фильтрации (user-based подход). Будем использовать датасет с фильмами MovieLens

1. Загрузка данных

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-04-07 16:41:23--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 173.194.218.139, 173.194.218.101, 173.194.218.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.218.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-04-07 16:41:23--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.97.132, 2607:f8b0:400c:c18::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.97.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.008s  

2024-04-07 16:41:24 (101 MB/s

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
movies = pd.read_csv('movies.csv') #информация о фильмах
ratings = pd.read_csv('ratings.csv') #рейтинги

2. Использование моделей из библиотеки Surprise

In [ ]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163008 sha256=a456f7c564cb8141c18cbc962cfae067cc16dbb6f8c55c633d611f87a2c4b6f8
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
from surprise import KNNBasic, SVD, SVDpp #возьмем три такие модели
from surprise import Dataset
from surprise import Reader

Объединим датафреймы и извлечем только нужную информацию для коллаборативной филльтрации: взаимодействие пользователей с фильмами - проставление рейтинга

In [ ]:
movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483


In [ ]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [ ]:
ratings.rating.min()

0.5

In [ ]:
ratings.rating.max()

5.0

In [ ]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

3. Кросс-валидация и вычисление метрики RMSE на трех разных моделях

In [ ]:
from surprise.model_selection import cross_validate

In [ ]:
algo1 = KNNBasic()
cross_validate(algo1, data, measures=['RMSE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9476  0.9493  0.9509  0.9461  0.9424  0.9473  0.0029  
Fit time          0.09    0.13    0.12    0.12    0.13    0.12    0.01    
Test time         1.44    1.26    1.44    1.29    1.43    1.37    0.08    


{'test_rmse': array([0.94760527, 0.9493067 , 0.95094495, 0.94608367, 0.94240484]),
 'fit_time': (0.09105181694030762,
  0.12971901893615723,
  0.12385010719299316,
  0.12478041648864746,
  0.1305403709411621),
 'test_time': (1.4449338912963867,
  1.2593252658843994,
  1.4378936290740967,
  1.2889459133148193,
  1.427304983139038)}

In [ ]:
algo2 = SVD()
cross_validate(algo2, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8807  0.8755  0.8706  0.8631  0.8742  0.8728  0.0059  
Fit time          2.23    1.61    2.12    1.54    1.55    1.81    0.30    
Test time         0.14    0.19    0.11    0.28    0.16    0.18    0.06    


{'test_rmse': array([0.88069702, 0.87553562, 0.87062493, 0.86306853, 0.87419992]),
 'fit_time': (2.232285499572754,
  1.607466697692871,
  2.116302728652954,
  1.5408473014831543,
  1.554117202758789),
 'test_time': (0.1430819034576416,
  0.1914505958557129,
  0.11187601089477539,
  0.2817511558532715,
  0.15828609466552734)}

In [ ]:
algo3 = SVDpp(cache_ratings=True)
cross_validate(algo3, data, measures=['RMSE'], cv=5, verbose=True)

Evaluating RMSE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8601  0.8648  0.8624  0.8610  0.8596  0.8616  0.0019  
Fit time          107.33  109.61  110.26  106.66  113.67  109.50  2.48    
Test time         12.30   12.50   12.30   12.39   12.64   12.43   0.13    


{'test_rmse': array([0.86012868, 0.86477386, 0.86239125, 0.8610232 , 0.85960634]),
 'fit_time': (107.33024501800537,
  109.60537433624268,
  110.25879573822021,
  106.66057753562927,
  113.66957998275757),
 'test_time': (12.303141832351685,
  12.49784779548645,
  12.301611185073853,
  12.386965274810791,
  12.639994144439697)}

Как видим, среднеквадратичной ошибки ниже 0,87 нам помогла добиться только модель SVDpp. При этом она прогружалась около 10 минут и заняла большее количество времени